In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from tensorflow.keras.layers import concatenate
from deepforest import CascadeForestClassifier
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5.csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images.shape)

1000
(1000, 320, 320, 9)


In [3]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)
print(trainImagesX.shape)
print(trainy.shape)

(800, 2)
(200, 2)
(800,)
(200,)
(800, 320, 320, 9)
(800,)


In [4]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [5]:

input = Input(shape = (320,320,9))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
    x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
x = tf.keras.layers.Dropout(0.4)(x)
output= Flatten()(x)
output = Dense (256, activation = "relu")(output)
output = Dense (1, activation = "sigmoid")(output)
model = Model(inputs=input, outputs=output)
# model.fit()


In [6]:
#mobilnet = create_cnn_mobilenet()
#mobilenet.compile(loss=None)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss=tf.losses.BinaryCrossentropy(),metrics = ['accuracy']) 
print(model.input)
print(model.output)
print(trainy.shape)
# mobilenetPredict = mobilenet.predict(trainImagesX)
# print(mobilenetPredict.shape)


KerasTensor(type_spec=TensorSpec(shape=(None, 320, 320, 9), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_1/Sigmoid:0', description="created by layer 'dense_1'")
(800,)


In [7]:
hist = model.fit(x=trainImagesX, y=trainy,validation_data=(testImagesX, testy),epochs=2)

Epoch 1/2


25/25 [==============================] - 53s 2s/step - loss: 0.8393 - accuracy: 0.6100 - val_loss: 0.6943 - val_accuracy: 0.4950
Epoch 2/2
25/25 [==============================] - 45s 2s/step - loss: 0.2498 - accuracy: 0.9162 - val_loss: 0.7240 - val_accuracy: 0.4950


In [8]:
performance = model.predict(testImagesX)
performance.round()
actual = []
for value in performance: 
    # print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

acutal = np.array(actual)
# print(actual)
# print(testy)
accuracy = accuracy_score(testy, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

7/7 [==============================] - 3s 412ms/step
Accuracy: 0.495000
Precision: 0.495000
Recall: 1.000000
F1 score: 0.662207
